#Pre Setup

##Imports lib and connect with Drive

In [ ]:
# Import các library, classs và hàm cần thiết

import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim

# Cài và import wandb
!pip install wandb
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# # Cài kaggle và tải data

# !pip install kaggle
# !mkdir ~/.kaggle

# # Upload file kaggle.json vào Colab, rồi chạy:

# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# # Download dataset CelebA

# !kaggle datasets download -d jessicali9530/celeba-dataset

# # Giải nén

# !unzip celeba-dataset.zip -d /content/celeba

##Setup Drive

In [ ]:
# Định nghĩa root để ảnh
celeba_root = '/content/celeba/img_align_celeba/'
print(os.listdir(celeba_root)[:10])

['img_align_celeba']


##Add Arguments


In [ ]:
# Thiết lập các tham số cho mô hình GAN

parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.001, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=224, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=100, help="interval between image sampling")
opt = parser.parse_args(args=[])
print(opt)

Namespace(n_epochs=200, batch_size=64, lr=0.001, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, img_size=224, channels=3, sample_interval=100)


##Wandb Initialization

In [ ]:
wandb.finish()
wandb.init(
    project = "GAN 2",
    name = "Emoji Creation 2",
    config = opt,
)

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss/D,▂▁▅▆▂▂█▃▄▃▃▄▄▃▃▂▃▂▃▂▂▂▃▂▂▁▃▂▃▄▂▃▃▂▂▃▂▁▂▄
loss/G,▅▇█▃▁██▇▇▅▆▅▅▆▆▆▅▆▆▆▅▅▆▆▇▇▇▆▅▅▅▇▆▆▆▆▆▆▆▅
epoch,0
loss/D,0.69485
loss/G,0.69533


##Environment

In [ ]:
# tạo thư mục images để lưu các ảnh kết quả

os.makedirs("images", exist_ok=True)

# Thiết lập kích thước ảnh và check xem có sử dụng GPU hay không

img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

#Model Definition

##Weight Init

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

##Generator

In [ ]:
# Định nghĩa class Generator (Máy biến nhiễu thành ảnh)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

##Discriminator

In [ ]:
# Định nghĩa class Discriminator (Thám tử phân biệt)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

#Traning Preparation

##Loss and Init

In [ ]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Khởi tạo generator and discriminator và thêm wandb để track
generator = Generator()
discriminator = Discriminator()

wandb.watch(generator, log="all", log_freq=200)
wandb.watch(discriminator, log="all", log_freq=200)

# Chuyển mô hình sang sử dụng GPU nếu có
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

##Prep Data

In [ ]:
# Chuẩn bị và nạp dữ liệu MNIST

transform=transforms.Compose(
            [
                transforms.Resize(opt.img_size),
                transforms.CenterCrop(opt.img_size),
                transforms.ToTensor(),
                transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
              ]
        )

dataset = ImageFolder(root=celeba_root, transform=transform)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=cuda,
)

##Optimizer

In [ ]:
# Định nghĩa và cấu hình Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr*0.1, betas=(opt.b1, opt.b2))

##Tensor Setup

In [ ]:
# Định dạng Tensor sử dụng GPU nếu có và device type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

#Traning Process

In [ ]:
# Tạo noise cố định để theo dõi chất lượng qua các epoch
z_fixed = torch.randn(25, opt.latent_dim).type(Tensor)

# Vòng lặp training
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Lưu ảnh mẫu sau mỗi batches được xử lí
        batches_done = epoch * len(dataloader) + i

        # Adversarial ground truths
        # Chuẩn bị nhãn và dữ liệu trên device
        valid = torch.full((imgs.size(0), 1), 1.0, device=device)
        fake  = torch.zeros((imgs.size(0), 1), device=device)
        real_imgs = imgs.to(device)

        # -----------------
        #  Train Generator
        # -----------------

        # Xóa gradient cũ
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        # Backpropagation
        g_loss.backward()

        # Cập nhật trọng số của generator
        optimizer_G.step()


        # ---------------------
        #  Train Discriminator
        # ---------------------

        if batches_done % 5 == 0:
          # Xóa gradient cũ
          optimizer_D.zero_grad()

          # Measure discriminator's ability to classify real from generated samples
          # Đánh giá trên ảnh thật và giả
          real_loss = adversarial_loss(discriminator(real_imgs), valid)
          fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
          #Tính trung bình của 2 loss
          d_loss = (real_loss + fake_loss) / 2

          # Backpropagation và cập nhật trọng số
          d_loss.backward()
          optimizer_D.step()

        # In ra thông tin về quá trình huẩn luyện
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        # Wandb log loss
        wandb.log(
            {
                "loss/D": d_loss.item(),
                "loss/G": g_loss.item(),
                "epoch": epoch
            },
            step=epoch * len(dataloader) + i,
        )

        if batches_done % opt.sample_interval == 0:
            # Lưu ảnh vào local storage
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            # Log grid của ảnh được gen hiện tại
            gen_imgs_vis = (gen_imgs + 1) / 2  # [-1,1] -> [0,1]
            # gen_imgs_vis = gen_imgs
            grid_current = make_grid(gen_imgs_vis[:25], nrow=5)
            wandb.log({
                "samples/current_batch": wandb.Image(grid_current, caption=f"step {batches_done}")
            }, step=batches_done)

            # 4.3) Log grid ảnh từ z cố định (giúp so sánh chất lượng qua thời gian)
            with torch.no_grad():
                gen_fixed = generator(z_fixed)
            gen_fixed_vis = (gen_fixed + 1)/2
            # gen_fixed_vis = gen_fixed
            grid_fixed = make_grid(gen_fixed_vis, nrow=5)
            wandb.log({
                "samples/fixed_noise": wandb.Image(grid_fixed, caption=f"fixed z @ step {batches_done}")
            }, step=batches_done)
            print("log")

[Epoch 0/200] [Batch 0/3166] [D loss: 0.693161] [G loss: 0.696238]
log
[Epoch 0/200] [Batch 1/3166] [D loss: 0.693161] [G loss: 0.695865]
[Epoch 0/200] [Batch 2/3166] [D loss: 0.693161] [G loss: 0.695778]
[Epoch 0/200] [Batch 3/3166] [D loss: 0.693161] [G loss: 0.695691]
[Epoch 0/200] [Batch 4/3166] [D loss: 0.693161] [G loss: 0.695479]
[Epoch 0/200] [Batch 5/3166] [D loss: 0.693450] [G loss: 0.695209]
[Epoch 0/200] [Batch 6/3166] [D loss: 0.693450] [G loss: 0.694749]
[Epoch 0/200] [Batch 7/3166] [D loss: 0.693450] [G loss: 0.694280]
[Epoch 0/200] [Batch 8/3166] [D loss: 0.693450] [G loss: 0.694148]
[Epoch 0/200] [Batch 9/3166] [D loss: 0.693450] [G loss: 0.693388]
[Epoch 0/200] [Batch 10/3166] [D loss: 0.694610] [G loss: 0.692387]
[Epoch 0/200] [Batch 11/3166] [D loss: 0.694610] [G loss: 0.692077]
[Epoch 0/200] [Batch 12/3166] [D loss: 0.694610] [G loss: 0.691586]
[Epoch 0/200] [Batch 13/3166] [D loss: 0.694610] [G loss: 0.691101]
[Epoch 0/200] [Batch 14/3166] [D loss: 0.694610] [G lo